In [81]:
# ссылка на репозиторий: https://github.com/AnastasiaKoshkina/ML
# доступ публичный, так как непонятно, каким пользователям гитхаба и/или на какие почты давать доступ

#подгружаем pandas
import pandas as pd 
#подгружаем numpy - понадобится для приведения дат к месяцам
import numpy as np 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [82]:
#ЗАДАЧА 1

#загружаем первый файл с информацией о посетителях
peopleData = pd.read_csv("CASC_Constant.csv", sep = ",")
peopleData = peopleData.drop(columns=["ActivationDate","SubscribedEmail","SubscribedPush"]) #удаление лишних столбцов
print(peopleData.info())
print(peopleData.head())


#загружаем второй файл с информацией о транзакциях
transactions = pd.read_csv("casc-resto-small.csv", sep = ",")
print(transactions.info())
print(transactions.head())
print(transactions.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerId  10000 non-null  int64  
 1   Age         9953 non-null   float64
 2   Sex         8874 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB
None
   CustomerId   Age     Sex
0     2728183  21.0  Female
1     2728198  21.0  Female
2     2728306  21.0  Female
3     2728178  22.0    Male
4     2728322  22.0    Male
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            1000 non-null   int64  
 1   Restaurant            1000 non-null   int64  
 2   RKDate                1000 non-null   object 
 3   RegionName            1000 non-null   object 
 4   BrandsNames           1000 non-null

In [83]:
#ЗАДАЧА 2

transactions.RKDate = pd.to_datetime(transactions.RKDate) # преобразование из строки в дату - т.к. дата в датасете была строкового типа
print(type(transactions.RKDate[0]))
transactions = transactions.sort_values("RKDate") # сортировка по возрастанию
print(transactions.tail())

transactionsID = transactions.groupby('CustomerID') # groupby по ID клиента

for index, item in transactions.iterrows():  # расчет RFM переменных и расчет переменной Y 
    
    #visithappen
    visit = 0
    for index, item in transactionsID.get_group(item['CustomerID']).iterrows():
        if pd.to_datetime("2017-12-31") >= item['RKDate'] >= pd.to_datetime("2017-07-01"):
            visit = 1 #если визит был в указанном диапазоне - возвращаем True. Иначе - False    
    transactions.loc[index,'Visithappen'] = visit
    
    #recency
    
    correct_date = transactionsID.get_group(item['CustomerID'])[transactionsID.get_group(item['CustomerID'])['RKDate'] < pd.to_datetime("2017-07-01")]
    if not correct_date.empty:
        recency = (pd.to_datetime("2017-07-01") - correct_date["RKDate"].max()) / np.timedelta64(1,'D')
    else:
        recency = 0 
    transactions.loc[index,'Recency'] = recency
    
    #frequency
    
    correct_date = transactionsID.get_group(item['CustomerID'])[transactionsID.get_group(item['CustomerID'])['RKDate'] < pd.to_datetime("2017-07-01")]
    if not correct_date.empty:
        months = (pd.to_datetime("2017-07-01") - correct_date["RKDate"].min()) / np.timedelta64(1,'M')
        visits = len(correct_date.index) 
        frequency = visits / months
    else:
        frequency = 0
    transactions.loc[index,'Frequency'] = frequency
    
    #monetvalue
    
    count = 0 
    money = 0
    for index, item in transactionsID.get_group(item['CustomerID']).iterrows():
        if item['RKDate'] < pd.to_datetime("2017-07-01"):
            count += 1
            money += item['SummBasic']
    if count == 0:
        monetvalue = 0
    if monetvalue != 0:
        monetvalue = money / count  
    count = 0 
    money = 0 
    transactions.loc[index,'MonetValue'] = monetvalue
    
print(transactions.info())
    
transactions = transactions.drop_duplicates(subset=['CustomerID'])  # избавление от повторов


transactions = transactions.drop(columns=["RKDate", "Restaurant", "RegionName","BrandsNames", "DishCategoryName", "Quantity", "SummBasic", "SummAfterPointsUsage"])  # удаление лишних столбцов

print(transactions.info())

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
     CustomerID  Restaurant     RKDate                    RegionName  \
209     2767598         450 2017-11-22  Москва и Московская область    
773     2768840         183 2017-11-29                   Новосибирск   
717     2821271         129 2017-12-28  Москва и Московская область    
803     2750656         295 2017-12-29  Москва и Московская область    
849     2748926         390 2017-12-30  Москва и Московская область    

    BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
209   Мама Раша             FOOD         1          9                  7.00  
773    IL Патио      NON ALCOHOL         1         10                 10.00  
717    IL Патио             FOOD         1         10                 10.00  
803      Шикари             FOOD         1         10                  6.87  
849    IL Патио             FOOD         1         10                 10.00  
<class 'pandas.core.frame.DataFrame'>
Int64Index

In [84]:
# ЗАДАНИЕ 3 
#добавление данных из второй таблицы

peopleData.rename(columns={'CustomerId': 'CustomerID'}, inplace=True)    # переименовываем столбец – изначально они не совпадают 
MainData = transactions.merge(peopleData) # соединяем обе таблицы 
MainData = MainData.dropna() # убираем пустые значения -  не все клиенты указали гендер
MainData = MainData.replace({'Sex': {'Male': True,'Female': False}}) # Меняем гендеры на тип bool

print(MainData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 1 to 461
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CustomerID   303 non-null    int64  
 1   Visithappen  303 non-null    float64
 2   Recency      303 non-null    float64
 3   Frequency    303 non-null    float64
 4   MonetValue   303 non-null    float64
 5   Age          303 non-null    float64
 6   Sex          303 non-null    bool   
dtypes: bool(1), float64(5), int64(1)
memory usage: 16.9 KB
None


In [85]:
# ЗАДАНИЕ 4 
#деление на тестовую и обучающую выборки
trg = MainData['Visithappen']  # Отделяем от выборки значение зависимой переменной
trn = MainData.drop(columns=['Visithappen'], axis=1)
Xtrn, Xtest, Ytrn, Ytest = train_test_split(trn, trg, test_size = 0.2) # Разбиваем выборку на тестовую и обучающую

print(Xtrn.head())
Ytrn

     CustomerID  Recency  Frequency  MonetValue   Age    Sex
236     2731279    751.0   0.040528         0.0  40.0   True
26      2756505    886.0   0.034353         0.0  36.0  False
92      2805934    850.0   0.035808         0.0  27.0   True
393     2846187    410.0   0.074236         0.0  28.0  False
214     2888593    769.0   0.039580         0.0  28.0  False


236    0.0
26     0.0
92     0.0
393    0.0
214    0.0
      ... 
181    0.0
165    0.0
132    0.0
402    0.0
177    0.0
Name: Visithappen, Length: 242, dtype: float64

In [86]:
# ЗАДАНИЕ 5
# обучение модели

logistic = LogisticRegression()
logistic.fit(Xtrn, Ytrn) # обучение модели
logistic.score(Xtrn, Ytrn) # оценка модели

1.0

In [87]:
# ЗАДАНИЕ 6 
#предсказывание Y

prediction = pd.Series(logistic.predict(Xtest)) # Предсказывание значения у и конвертация
prediction = prediction.rename('Visithappen') # переименовение, выборки хранились в одинаковом виде

In [88]:
# ЗАДАНИЕ 7
# подсчет метрики

report = classification_report(Ytest, prediction) # Считаем метрики модели
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        60
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00        61
   macro avg       1.00      1.00      1.00        61
weighted avg       1.00      1.00      1.00        61

